# Corpus Merging.

## CSAT 만들기

CSAT의 경우, 자체적인 POS 값들 달기 애매하기 때문에 E-lexicon proj에서 제공하는 POS 값을 우선적으로 가져와서 붙여 넣자. 아마 대부분 있지 않을까?

### Module Import

In [2]:
import pandas as pd # 데이터 처리
import numpy as np # 데이터 처리
import seaborn as sns # 데이터 시각화
from tqdm import tqdm # 데이터 처리
import re # 정규식으로 특수기호 및 char 처리
import json # 데이터 가공 후 저장할 형식
from typing import Any, Dict, List  # python data-type 정의
from pathlib import Path # 파일 경로 처리
import os # 파일 경로 및 처리
from json_handler import JsonFileHandler
from preprocessor import Preprocessor, TargetCorpusPreprocessor

### file path setting

In [10]:
# 파일 경로 확인.
proj_path = os.path.dirname(os.path.dirname(Path.cwd()))
corpora_dir = os.path.join(proj_path, 'Corpora')
CSAT_json_path = save_path = os.path.join(corpora_dir, 'CSAT-latest')

# 파일 경로 유효성 확인.
if not os.path.exists(proj_path):
    raise FileNotFoundError(f"Project path not found at {proj_path}")
if not os.path.exists(CSAT_json_path):
    raise FileNotFoundError(f"CSAT json path not found at {CSAT_json_path}")
if not any(file.endswith('.json') for file in os.listdir(CSAT_json_path)):
    raise FileExistsError(f"Any .json file doesn't exist in {CSAT_json_path}")
print("All file paths are valid.")

All file paths are valid.


### Utils: generate_word_list
중복을 허용하여 단어를 extend 한 list 생성.
이후 pandas 다른 method(ex: value_count)를 사용해서 빈도수 계산.

In [14]:
def generate_word_list(document_list: list, document_dir: str):
    # article 별 단어 분석.
    word_list = []   # word 정보를 담을 리스트
    json_handler = JsonFileHandler()

    for document_name in document_list: # corpus에서 하나의 json 파일을 가져와서
        document = json_handler.load_data(os.path.join(document_dir, document_name))  # 하나의 json 파일 내에 있는 모든 파일을 가져옴.

        if isinstance(document, list):   # 한 json 내에 여러 obj가 있는 경우, 
            for sentence in document:  # json 파일 내 있는 여러 문장들 수 
                word_list.extend(sentence['tokens'])
        else:   # 한 json 내에 하나의 obj가 있는 경우, 
            word_list.extend(document['tokens'])   

    return word_list

### CSAT(only): 수능 + 모의고사

In [ ]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
reading_corpus = [file for file in corpus_list if 'reading' in file]
listening_corpus = [file for file in corpus_list if 'listening' in file]

len(reading_corpus), len(listening_corpus)

(52, 49)

In [40]:
listening_word_list = generate_word_list(listening_corpus, CSAT_path)
reading_word_list = generate_word_list(reading_corpus, CSAT_path)
CSAT_only_word_list = pd.Series(listening_word_list + reading_word_list)

In [41]:
CSAT_only_word_list.shape

(58328,)

In [42]:
CSAT_only_word_list.value_counts()

.              3435
,              2787
the            2748
to             1627
of             1376
               ... 
substantial       1
tolls             1
merchants         1
immediacy         1
doubly            1
Name: count, Length: 7282, dtype: int64

In [45]:
CSAT_only_word_list_df = CSAT_only_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
CSAT_only_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only.xlsx'), index=False)
CSAT_only_word_list_df

,Word,Freq
0,.,3435
1,",",2787
2,the,2748
3,to,1627
4,of,1376
...,...,...
7277,substantial,1
7278,tolls,1
7279,merchants,1
7280,immediacy,1


### textbook(only)

In [12]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
textbook_corpus = [file for file in corpus_list if 'textbook' in file]

len(textbook_corpus)

71

In [46]:
textbook_word_list = generate_word_list(textbook_corpus, CSAT_path)
textbook_only_word_list = pd.Series(textbook_word_list)

In [47]:
textbook_only_word_list.shape

(9043,)

In [48]:
textbook_only_word_list.value_counts()

.            530
the          401
,            398
to           254
and          240
            ... 
finale         1
shoulders      1
teammates      1
field          1
enjoyed        1
Name: count, Length: 2076, dtype: int64

In [49]:
textbook_only_word_list_df = textbook_only_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
textbook_only_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only.xlsx'), index=False)
textbook_only_word_list_df

,Word,Freq
0,.,530
1,the,401
2,",",398
3,to,254
4,and,240
...,...,...
2071,finale,1
2072,shoulders,1
2073,teammates,1
2074,field,1


### CSAT Total

In [13]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
reading_corpus = [file for file in corpus_list if 'reading' in file]
listening_corpus = [file for file in corpus_list if 'listening' in file]
textbook_corpus = [file for file in corpus_list if 'textbook' in file]

len(reading_corpus), len(listening_corpus), len(textbook_corpus)

(52, 49, 71)

In [ ]:
listening_word_list = generate_word_list(listening_corpus, CSAT_path)
reading_word_list = generate_word_list(reading_corpus, CSAT_path)
textbook_word_list = generate_word_list(textbook_corpus, CSAT_path)
CSAT_word_list = pd.Series(listening_word_list + reading_word_list + textbook_word_list)

In [23]:
CSAT_word_list.shape

(67371,)

In [26]:
CSAT_word_list.value_counts()

.               3965
,               3185
the             3149
to              1881
of              1561
                ... 
reconnects         1
trolly             1
conveniently       1
tagging            1
treasured          1
Name: count, Length: 7796, dtype: int64

In [50]:
CSAT_word_list_df = CSAT_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
CSAT_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT.xlsx'), index=False)
CSAT_word_list_df

,Word,Freq
0,.,3965
1,",",3185
2,the,3149
3,to,1881
4,of,1561
...,...,...
7791,reconnects,1
7792,trolly,1
7793,conveniently,1
7794,tagging,1


## E-lexicon proj

E-lexicon proj를 보면, a의 orthographic N이 1이다. 즉, b,c,d,e 같이 치환된 케이스 중 비단어는 제외했다는 말이니, CSAT도 그대로 사용하기는 무리. 따라서 E-lexicon이랑 intersection을 먼저 시켜봐야 할 듯.

In [32]:
E_lexicon_proj_path = os.path.join(corpora_dir, 'E-lexicon-proj')

In [34]:
E_lexicon_df = pd.read_csv(os.path.join(E_lexicon_proj_path, 'English Lexicon Project Items original.csv'))
E_lexicon_df.head()

,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS
0,a,1,"10,610,626",16.177,"20,415.270",6.018,99.930,3.923,1,1.450,minor|NN
1,aah,3,222,5.403,52.710,3.430,7.560,2.803,2,1.850,#
2,Aaron,5,"10,806",9.288,14.650,2.874,1.930,2.212,3,1.850,NN
3,aback,5,387,5.958,0.290,1.204,0.180,1.204,0,1.950,RB
4,abacus,6,513,6.240,0.240,1.114,0.120,1.041,0,2.900,NN


### CSAT_edit

In [52]:
CSAT_only_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only.xlsx'))
textbook_only_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only.xlsx'))
CSAT_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT.xlsx'))

# overlap 1562개
CSAT_df.shape, CSAT_only_df.shape,textbook_only_df.shape

((7796, 2), (7282, 2), (2076, 2))

In [82]:
CSAT_only_POS_df = pd.merge(CSAT_only_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
CSAT_only_POS_df.dropna(subset=['POS']).reset_index(drop=True)

,Word,Freq,POS
0,the,2748,minor
1,to,1627,minor
2,of,1376,minor
3,and,1253,minor
4,a,1098,minor|NN
...,...,...,...
6264,substantial,1,JJ
6265,tolls,1,NN|VB
6266,merchants,1,NN
6267,immediacy,1,NN


In [83]:
textbook_only_POS_df = pd.merge(textbook_only_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
textbook_only_POS_df.dropna(subset=['POS']).reset_index(drop=True)

,Word,Freq,POS
0,the,401,minor
1,to,254,minor
2,and,240,minor
3,a,186,minor|NN
4,of,185,minor
...,...,...,...
1840,finale,1,NN
1841,shoulders,1,NN|VB
1842,teammates,1,NN
1843,field,1,NN|VB


각각 E-lexicon이랑 합치고 POS 달기

In [ ]:
CSAT_POS_df = pd.merge(CSAT_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
CSAT_POS_df.dropna(subset=['POS']).reset_index(drop=True)

,Word,Freq,POS
0,the,3149,minor
1,to,1881,minor
2,of,1561,minor
3,and,1493,minor
4,a,1284,minor|NN
...,...,...,...
6619,palms,1,NN|VB
6620,renewing,1,VB
6621,conveniently,1,RB
6622,tagging,1,VB


In [84]:
CSAT_only_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only_POS.xlsx'), index=False)
textbook_only_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only_POS.xlsx'), index=False)
CSAT_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_POS.xlsx'), index=False)

### HAL

### SUBTLEX

## HAL + SUBTLEX

## CSAT + HAL

## CSAT + SUBTLEX